In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
BATCH_SIZE = 64
STEPS = len(df)//BATCH_SIZE

In [ ]:
len(df)

In [ ]:
ohe_cols = ['cp']

In [ ]:
def preprocess_dataframe(df, ohe_cols):
    df = df.copy()
    columns = [pd.get_dummies(df[col]) for col in ohe_cols]
    df = pd.concat([df,*columns], axis=1)
    df = (df - df.min())/(df.max()-df.min())
    return df



def dataframe_to_dataset(df, ohe_cols, shuffle=True):
    df = preprocess_dataframe(df, ohe_cols)
    labels = df.pop('output')
    ds = tf.data.Dataset.from_tensor_slices((df.values,labels.values))
    ds = ds.map(lambda data, label: (data, tf.expand_dims(label,-1)))
    if shuffle:
        ds = ds.shuffle(303)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

X_train, X_val = train_test_split(df, stratify=df['output'])



train_dataset = dataframe_to_dataset(X_train, ohe_cols)
test_dataset = dataframe_to_dataset(X_val, ohe_cols)


for i in train_dataset.unbatch().take(1):
    print(i[0])

In [ ]:
dfp = preprocess_dataframe(df,['cp'])

In [ ]:

inp = keras.layers.Input(shape = (17))
x = keras.layers.Dense(125)(inp)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(75)(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(25)(x)
x = keras.layers.Dense(10)(x)
output = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inp, outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
callbacks = [
    #keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=50, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint('./heart_model.h5', save_best_only=True),
    keras.callbacks.ReduceLROnPlateau(factor=0.8, patience=10)
]


history = model.fit(train_dataset.repeat(),epochs=200, steps_per_epoch=STEPS, validation_data=test_dataset, callbacks=callbacks)


In [ ]:
model.evaluate(test_dataset)